In [1]:
!pip install boto3


In [2]:
import boto3
import os

In [3]:
from pathlib import Path

aws_access_key_id =os.getenv("AWS_ACCESS_ID")
aws_secret_access_key = "6os7o+kr8eVGS1Mqxrvo57UPlhFY3Yag9IDswbc4"

s3_client = boto3.client(
    "s3",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)
bucket_name =os.getenv("AWS_BUCKET_NAME")
prefix = os.getenv("AWS_BUCKET_PREFIX")  

files= []
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
files =[]
if "Contents" in response:
    files = [obj["Key"] for obj in response["Contents"]]
    print("Archivos disponibles:", files)
else:
    print("No se encontraron archivos en el dataset.")
    
download_path =  os.path.join("./docs/dataset/")
for file in files:
    document=file.split("/")[-1]
    document_path=os.path.join(download_path,document)

    if not os.path.isfile(document_path) and document_path.endswith(".pdf"):
        file_name = os.path.join(download_path, document)
        print(file_name)
        s3_client.download_file(bucket_name, file, file_name)
        print(f"Descargado: {file}")

Archivos disponibles: ['queplan_insurance/', 'queplan_insurance/POL120190177.pdf', 'queplan_insurance/POL320130223.pdf', 'queplan_insurance/POL320150503.pdf', 'queplan_insurance/POL320180100.pdf', 'queplan_insurance/POL320190074.pdf', 'queplan_insurance/POL320200071.pdf', 'queplan_insurance/POL320200214.pdf', 'queplan_insurance/POL320210063.pdf', 'queplan_insurance/POL320210210.pdf']


In [14]:
import fitz  # PyMuPDF propuesta
import os

pdf_folder = os.path.dirname("./docs/dataset/") ### Vamos a extraer el texto de los PDFs
def extract_text_from_pdf(pdf_path):
    """ Extrae texto de un archivo PDF """
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text("text") for page in doc])
    return text

# Extraemos el texto de todos los PDFs
pdf_texts = {}
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file)
        pdf_texts[file] = extract_text_from_pdf(pdf_path)

# Text example
for pdf, text in pdf_texts.items():
    print(f"\n📄 {pdf} (Primeros 500 caracteres):\n{text[:500]}")
    break


📄 POL120190177.pdf (Primeros 500 caracteres):
PÓLIZA DE ACCIDENTES PERSONALES / REEMBOLSO GASTOS MÉDICOS
Incorporada al Depósito de Pólizas bajo el código POL120190177
ARTÍCULO 1°: REGLAS APLICABLES AL CONTRATO
Se aplicarán al presente contrato de seguro las disposiciones contenidas en los artículos siguientes y las
normas legales de carácter imperativo establecidas en el título VIII, del Libro II, del Código de Comercio. Sin
embargo, se entenderán válidas las estipulaciones contractuales que sean más beneficiosas para el
asegurado o el ben


In [20]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

# Modelo de embeddings (uno liviano y eficiente)
#embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Dividimos texto en fragmentos
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)

# Convertimos textos en fragmentos
docs = []
for pdf_name, text in pdf_texts.items():
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        docs.append(Document(page_content=chunk, metadata={"source": pdf_name}))

# Convertir a embeddings y almacenar en FAISS
texts = [doc.page_content for doc in docs]
embeddings = embedding_model.encode(texts)

# Crear FAISS y guardar
#vector_db = FAISS.from_embeddings(embeddings, docs)

### 2nd AM
vector_db = FAISS.from_embeddings(
    [(doc.page_content, emb) for doc, emb in zip(docs, embeddings)],
    embedding_model
)

### 3rd
# vector_db = FAISS.from_texts(
#     texts, embedding_model, metadatas=[doc.metadata for doc in docs]
# )

vector_db.save_local("insurance_policies_db")

print("✅ Embeddings generated and stored in FAISS.")

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
### Embedings para nuestro documento Q&A pólizas
import json

# Nuestro Modelo de embeddings
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Cargamos las preguntas y respuestas que hicimos en equipo el documento Q&A
qa_file = "qna\qna_polizas.json"  # Cambia esto por la ubicación real del archivo

with open(qa_file, "r", encoding="utf-8") as file:
    qna_data = json.load(file)  # Formato esperado: [{"question": "...", "answer": "..."}, ...]

# Generamos los embeddings para cada pregunta
questions = [qa["question"] for qa in qna_data]
answers = [qa["answer"] for qa in qna_data]

# Convertimos las preguntas a embeddings
question_embeddings = embedding_model.encode(questions)

# Crear el documento de LangChain
docs = [Document(page_content=answers[i], metadata={"question": questions[i]}) for i in range(len(questions))]

# Lo almacenamos en FAISS
qna_db = FAISS.from_embeddings(
    [(doc.page_content, emb) for doc, emb in zip(docs, question_embeddings)],
    embedding_model
)

# Guardamos el doc generado de FAISS en disco
qna_db.save_local("insurance_qna_db")

print("✅ Embeddings de Base de datos FAISS con Q&A creada.")

Logged in as angelmorales01


In [ ]:
### Descargamos el modelo elegido en LOCAL a nuestra computadora
from huggingface_hub import snapshot_download

# model_id = "unsloth/Llama-3.2-3B"   # Aquí va el ID del modelo
# local_dir = "./llama3_3b"           # Aquí va el nombre de la carpeta donde se va a guardar
# model_id = "microsoft/phi-2"
# local_dir = "./microsoft_phi-2"
model_id = "HuggingFaceH4/zephyr-7b-alpha"
local_dir = "./zephyr-7b-alpha"

snapshot_download(repo_id=model_id, local_dir=local_dir, token=os.getenv("HUGGING_FACE_TOKEN")) # Aquí va tu token de Hugging Face

d:\anyoneIA\Proyecto final\InsurancePolicy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 34 files:   3%|▎         | 1/34 [00:00<00:11,  2.79it/s]

In [ ]:
### Esto es para fusionar tanto la base de datos de los PDFs como de nuestro documento de Q&A
from langchain.vectorstores import FAISS

# Cargamos la base de datos FAISS
vector_db_policies = FAISS.load_local("insurance_policies_db", embedding_model, allow_dangerous_deserialization=True,)
# Cargarmos la base de datos de Q&A propia
vector_db_qna = FAISS.load_local("insurance_qna_db", embedding_model, allow_dangerous_deserialization=True)

# Combinamos ambas bases FAISS en una sola
vector_db_policies.merge_from(vector_db_qna)

# Ahora vector_db contiene ambos conjuntos de datos
vector_db = vector_db_policies

print("✅ FAISS con documentos y Q&A cargado correctamente.")

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [ ]:
### Aquí generamos los tokens a partir del modelo descargado
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

#model_path = "./llama3_3b"  # Ruta donde descargaste el modelo
# model_path = "./microsoft_phi-2"  # Ruta donde descargaste el modelo
model_path = "./zephyr-7b-alpha"  # Ruta donde descargaste el modelo

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def retrieve_relevant_docs(query, top_k=3):
    query_embedding = embedding_model.encode(query)
    retrieved_docs = vector_db.similarity_search_by_vector(query_embedding, k=top_k)
    return retrieved_docs

In [ ]:
def format_prompt(query):
    relevant_docs = retrieve_relevant_docs(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    prompt = f"""Usa la siguiente información para responder de manera clara y precisa:

    {context}

    Pregunta del usuario: {query}
    """
    
    return prompt


In [ ]:
### Integración de SerpAPI GOOGLE Search OPCIONAL
#from langchain_community.tools import SerpAPIWrapper
from langchain_community.utilities import SerpAPIWrapper

search = SerpAPIWrapper(serpapi_api_key="AQUÍ VA TU TOKEN GENERALO AQUÍ --->")  # Obtén la clave en https://serpapi.com/

def search_web(query, num_results=3):
    results = search.run(query)
    return results[:num_results]

query = "Últimas noticias sobre seguros en América"
results = search_web(query)
print(results)

In [ ]:
### Prompt integrando búsquedas con GOOGLE
def format_prompt(query):
    relevant_docs = retrieve_relevant_docs(query)
    context = "\n\n".join([doc.page_content for doc in relevant_docs])

    # Buscamos en Google si no hay documentos relevantes
    if not context:
        context = search_web(query)

    prompt = f"""Usa la siguiente información para responder de manera clara y precisa:

    {context}

    Pregunta del usuario: {query}
    """

    return prompt

In [ ]:
### Aquí configuramos el texto por respuesta, temperatura, etc, a partir de nuestro modelo
import torch

def generate_response(query):
    prompt = format_prompt(query)
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
    model.to(inputs.input_ids.device)
    
    output = model.generate(**inputs, max_length=500, temperature=0.7)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    
    return response

c:\Users\AER\Downloads\AnyONE AI\00 Final Project\AnyOne3\lib\site-packages\transformers\generation\configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Usa la siguiente información para responder de manera clara y precisa:

    que aparece detallado con tal carácter en las Condiciones Particulares de la Póliza.

encuentre amparado por la póliza. Tampoco estarán cubiertos los tratamientos secundarios a las cirugías
indicadas.

encuentre amparado por la póliza. Tampoco estarán cubiertos los tratamientos secundarios a las cirugías
indicadas.

    Pregunta del usuario: ¿Qué se considera un Accidente según la póliza POL120190177?
    Respuesta: Un Accidente según la póliza POL120190177 es un Accidente de la póliza.

"""

# %%
# Imports
# -------

import re

from. import pd_utils
from. import pd_utils_exceptions
from. import pd_utils_pols
from. import pd_utils_pols_exceptions
from. import pd_utils_pols_exceptions_pols
from. import pd_utils_pols_exceptions_pols_exceptions
from. import pd_utils_pols_exceptions_pols_exceptions_pols
from. import pd_utils_pols_exceptions_pols_exceptions_pols_exceptions
from. import pd_utils_pols_exceptions_pols_

In [ ]:
### Aquí está la primera pregunta de nuestro PROMPT al RAG creado
query = "¿Qué se considera un Accidente según la póliza POL120190177?"
response = generate_response(query)
print(response)

In [ ]:
query = "¿Me podrías crear una póliza de seguro de hogar?"
response = generate_response(query)
print(response)

In [ ]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from ragas.evaluation import evaluate
from ragas.metrics import context_precision, context_recall
from datasets import Dataset

# Cargamos el modelo y tokenizador de Hugging Face
model_id = "unsloth/Llama-3.2-3B" # Aquí el nombre del modelo que estén usando
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Creamos el pipeline de generación de texto
text_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Integramos el modelo en LangChain
llm = HuggingFacePipeline(pipeline=text_pipeline)

# Aquí va nuestro feedback para medir y mejorar la precisión del modelo
data = Dataset.from_dict({
    "question": [query],
    "answer": [response],
    "contexts": [[docs[0].page_content]],
    "reference": ["Un Accidente es un suceso imprevisto, involuntario, repentino y fortuito, causado por medios externos y de modo violento, que afecta al organismo del asegurado, ocasionándole lesiones visibles o internas."]
})

# Evaluar con modelo de Hugging Face
#evaluation = evaluate(data, metrics=[context_precision, context_recall], llm=llm)
evaluation = evaluate(data, metrics=[context_precision, context_recall], llm=llm, batch_size=1)
print("\n📊 Evaluación de la respuesta:", evaluation)

Device set to use cpu
C:\Users\AER\AppData\Local\Temp\ipykernel_5940\2555293172.py:16: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline)


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Batch 1/2:   0%|          | 0/1 [00:00<?, ?it/s]

Exception raised in Job[0]: TimeoutError()
Exception raised in Job[1]: TimeoutError()



📊 Evaluación de la respuesta: {'context_precision': nan, 'context_recall': nan}


In [ ]:
import ragas.metrics
print(dir(ragas.metrics))

['AgentGoalAccuracyWithReference', 'AgentGoalAccuracyWithoutReference', 'AnswerCorrectness', 'AnswerRelevancy', 'AnswerSimilarity', 'AspectCritic', 'BleuScore', 'ContextEntityRecall', 'ContextPrecision', 'ContextRecall', 'ContextUtilization', 'DataCompyScore', 'DistanceMeasure', 'ExactMatch', 'FactualCorrectness', 'Faithfulness', 'FaithfulnesswithHHEM', 'InstanceRubrics', 'LLMContextPrecisionWithReference', 'LLMContextPrecisionWithoutReference', 'LLMContextRecall', 'LLMSQLEquivalence', 'Metric', 'MetricOutputType', 'MetricType', 'MetricWithEmbeddings', 'MetricWithLLM', 'MultiModalFaithfulness', 'MultiModalRelevance', 'MultiTurnMetric', 'NoiseSensitivity', 'NonLLMContextPrecisionWithReference', 'NonLLMContextRecall', 'NonLLMStringSimilarity', 'ResponseRelevancy', 'RougeScore', 'RubricsScore', 'SemanticSimilarity', 'SimpleCriteriaScore', 'SingleTurnMetric', 'StringPresence', 'SummarizationScore', 'ToolCallAccuracy', 'TopicAdherenceScore', '__all__', '__builtins__', '__cached__', '__doc__

In [ ]:
query = "¿Qué gastos no son cubiertos por la póliza POL120190177?"
response = generate_response(query)
print(response)

Usa la siguiente información para responder de manera clara y precisa:

    ocurridos durante el mismo año póliza. Al concretarse la renovación de la póliza, se establecerá una nueva
suma asegurada por asegurado, por año póliza, para los gastos incurridos por accidentes, enfermedades o
padecimientos cubiertos por la renovación en curso, así como a los gastos incurridos en esta nueva
vigencia, por accidentes, enfermedades o padecimientos cubiertos en las vigencias previas, aún para

Póliza, que, habiendo superado el Deducible, la compañía reembolsará al Asegurado Titular o, en su
defecto, a los herederos legales de éste, o pagará al Prestador, los Gastos Reembolsables durante la
vigencia de este contrato de seguro y en los términos y condiciones señalados en estas Condiciones
Generales, todo lo que, por su naturaleza, se indica en las Condiciones Particulares de la Póliza.

Póliza, que, habiendo superado el Deducible, la compañía reembolsará al Asegurado Titular o en su defecto,
a los h